In [ ]:
# we need the version of dask geopandas that has sjoin
!pip install git+https://github.com/geopandas/dask-geopandas.git

In [1]:
import pandas
import dask.dataframe
import dask
import geopandas
import dask_geopandas 

In [3]:
dask_geopandas.__version__

'v0.1.0a4+16.gf18ae2c'

## Signature geometry

In [4]:
signatures = geopandas.read_parquet("../../urbangrammar_samba/spatial_signatures/signatures/signatures_combined_levels_orig.pq")
signatures.head()

,kmeans10gb,geometry,level2,signature_type
0,0,"POLYGON Z ((62220.000 798500.000 0.000, 62110....",0.0,0_0
1,0,"POLYGON Z ((63507.682 796515.169 0.000, 63471....",0.0,0_0
2,0,"POLYGON Z ((65953.174 802246.172 0.000, 65950....",0.0,0_0
3,0,"POLYGON Z ((67297.740 803435.800 0.000, 67220....",0.0,0_0
4,0,"POLYGON Z ((75760.000 852670.000 0.000, 75700....",0.0,0_0


In [5]:
types = {
    "0_0": "Countryside agriculture",
    "1_0": "Accessible suburbia",
    "3_0": "Open sprawl",
    "4_0": "Wild countryside",
    "5_0": "Warehouse land",
    "6_0": "Gridded residential quarters",
    "7_0": "Urban buffer",
    "8_0": "Disconnected suburbia",
    "2_0": "Dense residential neighbourhoods",
    "2_1": "Connected residential neighbourhoods",
    "2_2": "Dense urban neighbourhoods",
    "9_0": "Local urbanity",
    "9_1": "Distilled urbanity",
    "9_2": "Regional urbanity",
    "9_4": "Metropolitan urbanity",
    "9_5": "Hyper distilled urbanity",
    "9_3": "outlier",
    "9_6": "outlier",
    "9_7": "outlier",
    "9_8": "outlier",
}

In [6]:
signatures["type"] = signatures.signature_type.map(types)
signatures["id"] = range(len(signatures))

In [7]:
signatures.head()

,kmeans10gb,geometry,level2,signature_type,type,id
0,0,"POLYGON Z ((62220.000 798500.000 0.000, 62110....",0.0,0_0,Countryside agriculture,0
1,0,"POLYGON Z ((63507.682 796515.169 0.000, 63471....",0.0,0_0,Countryside agriculture,1
2,0,"POLYGON Z ((65953.174 802246.172 0.000, 65950....",0.0,0_0,Countryside agriculture,2
3,0,"POLYGON Z ((67297.740 803435.800 0.000, 67220....",0.0,0_0,Countryside agriculture,3
4,0,"POLYGON Z ((75760.000 852670.000 0.000, 75700....",0.0,0_0,Countryside agriculture,4


In [8]:
signatures = signatures[["id", "type", "geometry"]]

In [9]:
signatures

,id,type,geometry
0,0,Countryside agriculture,"POLYGON Z ((62220.000 798500.000 0.000, 62110...."
1,1,Countryside agriculture,"POLYGON Z ((63507.682 796515.169 0.000, 63471...."
2,2,Countryside agriculture,"POLYGON Z ((65953.174 802246.172 0.000, 65950...."
3,3,Countryside agriculture,"POLYGON Z ((67297.740 803435.800 0.000, 67220...."
4,4,Countryside agriculture,"POLYGON Z ((75760.000 852670.000 0.000, 75700...."
...,...,...,...
96699,96699,outlier,"POLYGON ((323321.005 463795.416, 323319.842 46..."
96700,96700,outlier,"POLYGON ((325929.840 1008792.061, 325927.377 1..."
96701,96701,outlier,"POLYGON ((337804.770 1013422.583, 337800.122 1..."
96702,96702,outlier,"POLYGON ((422304.270 1147826.990, 422296.000 1..."


### Remove Z coordinates

In [10]:
import pygeos

In [11]:
signatures["geometry"] = pygeos.apply(signatures.geometry.values.data, lambda x: x, include_z=False)

In [12]:
signatures

,id,type,geometry
0,0,Countryside agriculture,"POLYGON ((62220.000 798500.000, 62110.000 7985..."
1,1,Countryside agriculture,"POLYGON ((63507.682 796515.169, 63471.097 7965..."
2,2,Countryside agriculture,"POLYGON ((65953.174 802246.172, 65950.620 8022..."
3,3,Countryside agriculture,"POLYGON ((67297.740 803435.800, 67220.289 8034..."
4,4,Countryside agriculture,"POLYGON ((75760.000 852670.000, 75700.000 8527..."
...,...,...,...
96699,96699,outlier,"POLYGON ((323321.005 463795.416, 323319.842 46..."
96700,96700,outlier,"POLYGON ((325929.840 1008792.061, 325927.377 1..."
96701,96701,outlier,"POLYGON ((337804.770 1013422.583, 337800.122 1..."
96702,96702,outlier,"POLYGON ((422304.270 1147826.990, 422296.000 1..."


In [23]:
signatures.to_file("spatial_signatures_GB.gpkg", driver="GPKG")

## Link data

In [31]:
from dask.distributed import Client, LocalCluster
client = Client(LocalCluster(n_workers=8))
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 125.55 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42233,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 125.55 GiB
Comm: tcp://127.0.0.1:43399,Total threads: 2
Dashboard: http://127.0.0.1:35317/status,Memory: 15.69 GiB
Nanny: tcp://127.0.0.1:45667,


In [32]:
tessellation = dask_geopandas.read_parquet("../../urbangrammar_samba/spatial_signatures/tessellation/tess_*.pq")

/opt/conda/lib/python3.8/site-packages/dask_geopandas/io/parquet.py:83: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  meta = geopandas.GeoDataFrame(meta, crs=crs)


In [33]:
tessellation = dask_geopandas.from_dask_dataframe(tessellation[["hindex", "tessellation"]]).set_geometry("tessellation").explode()

In [34]:
tessellation["point"] = tessellation.representative_point()
tessellation = tessellation.set_geometry("point")

In [35]:
joined = dask_geopandas.sjoin(tessellation, signatures, op="within")

/opt/conda/lib/python3.8/site-packages/dask_geopandas/sjoin.py:50: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:27700

  meta = geopandas.sjoin(left._meta, right._meta, how=how, op=op)


In [36]:
joined

,hindex,tessellation,point,index_right,id,type
npartitions=103,,,,,,
,object,geometry,geometry,int64,int64,object
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [37]:
form = dask.dataframe.read_parquet("../../urbangrammar_samba/spatial_signatures/morphometrics/convolutions/conv_*.pq")
function = dask.dataframe.read_parquet("../../urbangrammar_samba/spatial_signatures/functional/functional/func_*.pq")

In [38]:
data = dask.dataframe.multi.concat([function.reset_index(), form.reset_index(drop=True)], axis=1)
data = data.drop(columns=["keep_q1", "keep_q2", "keep_q3"])

/opt/conda/lib/python3.8/site-packages/dask/dataframe/multi.py:1212: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [39]:
data

,hindex,population_q1,population_q2,population_q3,night_lights_q1,night_lights_q2,night_lights_q3,"A, B, D, E. Agriculture, energy and water_q1","A, B, D, E. Agriculture, energy and water_q2","A, B, D, E. Agriculture, energy and water_q3",C. Manufacturing_q1,C. Manufacturing_q2,C. Manufacturing_q3,F. Construction_q1,F. Construction_q2,F. Construction_q3,"G, I. Distribution, hotels and restaurants_q1","G, I. Distribution, hotels and restaurants_q2","G, I. Distribution, hotels and restaurants_q3","H, J. Transport and communication_q1","H, J. Transport and communication_q2","H, J. Transport and communication_q3","K, L, M, N. Financial, real estate, professional and administrative activities_q1","K, L, M, N. Financial, real estate, professional and administrative activities_q2","K, L, M, N. Financial, real estate, professional and administrative activities_q3","O,P,Q. Public administration, education and health_q1","O,P,Q. Public administration, education and health_q2","O,P,Q. Public administration, education and health_q3","R, S, T, U. Other_q1","R, S, T, U. Other_q2","R, S, T, U. Other_q3",Code_18_124_q1,Code_18_124_q2,Code_18_124_q3,Code_18_211_q1,Code_18_211_q2,Code_18_211_q3,Code_18_121_q1,Code_18_121_q2,Code_18_121_q3,Code_18_421_q1,Code_18_421_q2,Code_18_421_q3,Code_18_522_q1,Code_18_522_q2,Code_18_522_q3,Code_18_142_q1,Code_18_142_q2,Code_18_142_q3,Code_18_141_q1,Code_18_141_q2,Code_18_141_q3,Code_18_112_q1,Code_18_112_q2,Code_18_112_q3,Code_18_231_q1,Code_18_231_q2,Code_18_231_q3,Code_18_311_q1,Code_18_311_q2,Code_18_311_q3,Code_18_131_q1,Code_18_131_q2,Code_18_131_q3,Code_18_123_q1,Code_18_123_q2,Code_18_123_q3,Code_18_122_q1,Code_18_122_q2,Code_18_122_q3,Code_18_512_q1,Code_18_512_q2,Code_18_512_q3,Code_18_243_q1,Code_18_243_q2,Code_18_243_q3,Code_18_313_q1,Code_18_313_q2,Code_18_313_q3,Code_18_412_q1,Code_18_412_q2,Code_18_412_q3,Code_18_321_q1,Code_18_321_q2,Code_18_321_q3,Code_18_322_q1,Code_18_322_q2,Code_18_322_q3,Code_18_324_q1,Code_18_324_q2,Code_18_324_q3,Code_18_111_q1,Code_18_111_q2,Code_18_111_q3,Code_18_423_q1,Code_18_423_q2,Code_18_423_q3,Code_18_523_q1,Code_18_523_q2,Code_18_523_q3,mean_q1,mean_q2,mean_q3,Code_18_312_q1,Code_18_312_q2,Code_18_312_q3,Code_18_133_q1,Code_18_133_q2,Code_18_133_q3,Code_18_333_q1,Code_18_333_q2,Code_18_333_q3,Code_18_332_q1,Code_18_332_q2,Code_18_332_q3,Code_18_411_q1,Code_18_411_q2,Code_18_411_q3,supermarkets_nearest,supermarkets_counts,listed_nearest,listed_counts,fhrs_nearest,fhrs_counts,culture_nearest,culture_counts,nearest_water,nearest_retail_centre,Code_18_132_q1,Code_18_331_q2,Code_18_222_q1,Code_18_511_q3,Code_18_242_q1,Code_18_511_q2,Code_18_242_q3,Code_18_331_q1,Code_18_334_q2,Code_18_511_q1,Code_18_334_q1,Code_18_222_q3,Code_18_242_q2,Code_18_244_q3,Code_18_521_q2,Code_18_334_q3,Code_18_244_q1,Code_18_244_q2,Code_18_331_q3,Code_18_132_q2,Code_18_132_q3,Code_18_521_q1,Code_18_222_q2,Code_18_521_q3,sdbAre_q1,sdbAre_q2,sdbAre_q3,sdbPer_q1,sdbPer_q2,sdbPer_q3,sdbCoA_q1,sdbCoA_q2,sdbCoA_q3,ssbCCo_q1,ssbCCo_q2,ssbCCo_q3,ssbCor_q1,ssbCor_q2,ssbCor_q3,ssbSqu_q1,ssbSqu_q2,ssbSqu_q3,ssbERI_q1,ssbERI_q2,ssbERI_q3,ssbElo_q1,ssbElo_q2,ssbElo_q3,ssbCCM_q1,ssbCCM_q2,ssbCCM_q3,ssbCCD_q1,ssbCCD_q2,ssbCCD_q3,stbOri_q1,stbOri_q2,stbOri_q3,sdcLAL_q1,sdcLAL_q2,sdcLAL_q3,sdcAre_q1,sdcAre_q2,sdcAre_q3,sscCCo_q1,sscCCo_q2,sscCCo_q3,sscERI_q1,sscERI_q2,sscERI_q3,stcOri_q1,stcOri_q2,stcOri_q3,sicCAR_q1,sicCAR_q2,sicCAR_q3,stbCeA_q1,stbCeA_q2,stbCeA_q3,mtbAli_q1,mtbAli_q2,mtbAli_q3,mtbNDi_q1,mtbNDi_q2,mtbNDi_q3,mtcWNe_q1,mtcWNe_q2,mtcWNe_q3,mdcAre_q1,mdcAre_q2,mdcAre_q3,ltcWRE_q1,ltcWRE_q2,ltcWRE_q3,ltbIBD_q1,ltbIBD_q2,ltbIBD_q3,sdsSPW_q1,sdsSPW_q2,sdsSPW_q3,sdsSWD_q1,sdsSWD_q2,sdsSWD_q3,sdsSPO_q1,sdsSPO_q2,sdsSPO_q3,sdsLen_q1,sdsLen_q2,sdsLen_q3,sssLin_q1,sssLin_q2,sssLin_q3,ldsMSL_q1,ldsMSL_q2,ldsMSL_q3,mtdDeg_q1,mtdDeg_q2,mtdDeg_q3,lcdMes_q1,lcdMes_q2,lcdMes_q3,linP3W_q1,linP3W_q2,linP3W_q3,linP4W_q1,linP4W_q2,linP4W_q3,linPDE_q1,linPDE_q2,linPDE_q3,lcnClo_q1,lcnClo_q2,lcnClo_q3,ldsCDL_q

In [40]:
merged = joined[["hindex", "id", "type"]].merge(data, on="hindex", how="left")

In [41]:
merged

,hindex,id,type,population_q1,population_q2,population_q3,night_lights_q1,night_lights_q2,night_lights_q3,"A, B, D, E. Agriculture, energy and water_q1","A, B, D, E. Agriculture, energy and water_q2","A, B, D, E. Agriculture, energy and water_q3",C. Manufacturing_q1,C. Manufacturing_q2,C. Manufacturing_q3,F. Construction_q1,F. Construction_q2,F. Construction_q3,"G, I. Distribution, hotels and restaurants_q1","G, I. Distribution, hotels and restaurants_q2","G, I. Distribution, hotels and restaurants_q3","H, J. Transport and communication_q1","H, J. Transport and communication_q2","H, J. Transport and communication_q3","K, L, M, N. Financial, real estate, professional and administrative activities_q1","K, L, M, N. Financial, real estate, professional and administrative activities_q2","K, L, M, N. Financial, real estate, professional and administrative activities_q3","O,P,Q. Public administration, education and health_q1","O,P,Q. Public administration, education and health_q2","O,P,Q. Public administration, education and health_q3","R, S, T, U. Other_q1","R, S, T, U. Other_q2","R, S, T, U. Other_q3",Code_18_124_q1,Code_18_124_q2,Code_18_124_q3,Code_18_211_q1,Code_18_211_q2,Code_18_211_q3,Code_18_121_q1,Code_18_121_q2,Code_18_121_q3,Code_18_421_q1,Code_18_421_q2,Code_18_421_q3,Code_18_522_q1,Code_18_522_q2,Code_18_522_q3,Code_18_142_q1,Code_18_142_q2,Code_18_142_q3,Code_18_141_q1,Code_18_141_q2,Code_18_141_q3,Code_18_112_q1,Code_18_112_q2,Code_18_112_q3,Code_18_231_q1,Code_18_231_q2,Code_18_231_q3,Code_18_311_q1,Code_18_311_q2,Code_18_311_q3,Code_18_131_q1,Code_18_131_q2,Code_18_131_q3,Code_18_123_q1,Code_18_123_q2,Code_18_123_q3,Code_18_122_q1,Code_18_122_q2,Code_18_122_q3,Code_18_512_q1,Code_18_512_q2,Code_18_512_q3,Code_18_243_q1,Code_18_243_q2,Code_18_243_q3,Code_18_313_q1,Code_18_313_q2,Code_18_313_q3,Code_18_412_q1,Code_18_412_q2,Code_18_412_q3,Code_18_321_q1,Code_18_321_q2,Code_18_321_q3,Code_18_322_q1,Code_18_322_q2,Code_18_322_q3,Code_18_324_q1,Code_18_324_q2,Code_18_324_q3,Code_18_111_q1,Code_18_111_q2,Code_18_111_q3,Code_18_423_q1,Code_18_423_q2,Code_18_423_q3,Code_18_523_q1,Code_18_523_q2,Code_18_523_q3,mean_q1,mean_q2,mean_q3,Code_18_312_q1,Code_18_312_q2,Code_18_312_q3,Code_18_133_q1,Code_18_133_q2,Code_18_133_q3,Code_18_333_q1,Code_18_333_q2,Code_18_333_q3,Code_18_332_q1,Code_18_332_q2,Code_18_332_q3,Code_18_411_q1,Code_18_411_q2,Code_18_411_q3,supermarkets_nearest,supermarkets_counts,listed_nearest,listed_counts,fhrs_nearest,fhrs_counts,culture_nearest,culture_counts,nearest_water,nearest_retail_centre,Code_18_132_q1,Code_18_331_q2,Code_18_222_q1,Code_18_511_q3,Code_18_242_q1,Code_18_511_q2,Code_18_242_q3,Code_18_331_q1,Code_18_334_q2,Code_18_511_q1,Code_18_334_q1,Code_18_222_q3,Code_18_242_q2,Code_18_244_q3,Code_18_521_q2,Code_18_334_q3,Code_18_244_q1,Code_18_244_q2,Code_18_331_q3,Code_18_132_q2,Code_18_132_q3,Code_18_521_q1,Code_18_222_q2,Code_18_521_q3,sdbAre_q1,sdbAre_q2,sdbAre_q3,sdbPer_q1,sdbPer_q2,sdbPer_q3,sdbCoA_q1,sdbCoA_q2,sdbCoA_q3,ssbCCo_q1,ssbCCo_q2,ssbCCo_q3,ssbCor_q1,ssbCor_q2,ssbCor_q3,ssbSqu_q1,ssbSqu_q2,ssbSqu_q3,ssbERI_q1,ssbERI_q2,ssbERI_q3,ssbElo_q1,ssbElo_q2,ssbElo_q3,ssbCCM_q1,ssbCCM_q2,ssbCCM_q3,ssbCCD_q1,ssbCCD_q2,ssbCCD_q3,stbOri_q1,stbOri_q2,stbOri_q3,sdcLAL_q1,sdcLAL_q2,sdcLAL_q3,sdcAre_q1,sdcAre_q2,sdcAre_q3,sscCCo_q1,sscCCo_q2,sscCCo_q3,sscERI_q1,sscERI_q2,sscERI_q3,stcOri_q1,stcOri_q2,stcOri_q3,sicCAR_q1,sicCAR_q2,sicCAR_q3,stbCeA_q1,stbCeA_q2,stbCeA_q3,mtbAli_q1,mtbAli_q2,mtbAli_q3,mtbNDi_q1,mtbNDi_q2,mtbNDi_q3,mtcWNe_q1,mtcWNe_q2,mtcWNe_q3,mdcAre_q1,mdcAre_q2,mdcAre_q3,ltcWRE_q1,ltcWRE_q2,ltcWRE_q3,ltbIBD_q1,ltbIBD_q2,ltbIBD_q3,sdsSPW_q1,sdsSPW_q2,sdsSPW_q3,sdsSWD_q1,sdsSWD_q2,sdsSWD_q3,sdsSPO_q1,sdsSPO_q2,sdsSPO_q3,sdsLen_q1,sdsLen_q2,sdsLen_q3,sssLin_q1,sssLin_q2,sssLin_q3,ldsMSL_q1,ldsMSL_q2,ldsMSL_q3,mtdDeg_q1,mtdDeg_q2,mtdDeg_q3,lcdMes_q1,lcdMes_q2,lcdMes_q3,linP3W_q1,linP3W_q2,linP3W_q3,linP4W_q1,linP4W_q2,linP4W_q3,linPDE_q1,linPDE_q2,linPDE_q3,lcnClo_q1,lcnClo_q2,lcnClo_q3,

In [42]:
per_geometry = merged.drop(columns=["hindex", "type"]).groupby("id").mean()
per_type = merged.drop(columns=["hindex", "id"]).groupby("type").mean()

In [43]:
%%time
per_type, per_geometry = dask.compute(per_type, per_geometry)

CPU times: user 2min 15s, sys: 16.5 s, total: 2min 31s
Wall time: 10min 59s


In [44]:
per_type

,population_q1,population_q2,population_q3,night_lights_q1,night_lights_q2,night_lights_q3,"A, B, D, E. Agriculture, energy and water_q1","A, B, D, E. Agriculture, energy and water_q2","A, B, D, E. Agriculture, energy and water_q3",C. Manufacturing_q1,...,lseCWA_q3,lteOri_q1,lteOri_q2,lteOri_q3,lteWNB_q1,lteWNB_q2,lteWNB_q3,lieWCe_q1,lieWCe_q2,lieWCe_q3
type,,,,,,,,,,,,,,,,,,,,,
Accessible suburbia,2.050706,3.379153,5.408408,9.489520,10.841694,12.479147,0.004403,0.008314,0.016678,0.033496,...,1024.211551,11.959011,18.969432,26.779136,0.002341,0.004509,0.008702,0.000316,0.001143,6.209091e-02
Connected residential neighbourhoods,2.768197,5.215322,10.150766,16.992753,19.461435,22.785900,0.004089,0.009717,0.024263,0.041320,...,634.888079,16.927334,25.560166,33.074361,0.003323,0.006636,0.012677,0.000241,0.000621,2.082159e-03
Countryside agriculture,0.488367,1.099359,2.227828,0.733194,1.114339,1.993260,0.014474,0.030804,0.063268,0.024958,...,2683.472166,11.800394,21.014110,30.758801,0.000778,0.001422,0.002867,0.000304,0.000688,6.008135e-02
Dense residential neighbourhoods,2.436451,5.246222,11.212626,17.809638,21.212227,25.795960,0.004034,0.011358,0.033611,0.050383,...,789.229822,8.640260,15.685687,24.590759,0.002760,0.005613,0.011617,0.000251,0.000651,1.526671e-03
Dense urban neighbourhoods,3.218808,8.325145,20.631266,26.575609,31.886952,39.234548,0.003898,0.015113,0.052371,0.068630,...,599.422922,12.430778,20.640303,29.274374,0.003234,0.006464,0.012999,0.000236,0.000626,1.275841e-03
Disconnected suburbia,2.525493,4.443459,7.988258,10.351643,11.992856,13.980314,0.005102,0.010256,0.021823,0.046251,...,1043.000190,19.080813,27.102402,33.898633,0.001904,0.003416,0.006554,0.000354,0.001548,3.811496e+06
Distilled urbanity,2.312175,14.157575,49.405958,84.951933,100.969925,129.018291,0.000227,0.136098,0.914424,0.270133,...,213.951096,10.913652,18.836472,28.241675,0.004830,0.009678,0.019067,0.000091,0.000531,1.211574e-03
Gridded residential quarters,2.175958,4.315135,9.148197,12.539161,14.616998,17.383908,0.003946,0.009692,0.024921,0.045200,...,607.162627,14.732847,22.629535,30.589887,0.004761,0.009697,0.019356,0.000336,0.000723,1.373951e-03
Hyper distilled urbanity,6.242168,21.600561,60.172323,94.615703,114.704710,162.775416,0.009578,0.342838,1.504627,0.992524,...,175.591052,17.799170,26.438240,33.971354,0.005597,0.011288,0.114848,0.000104,0.000566,1.200419e-03


In [45]:
per_geometry

,population_q1,population_q2,population_q3,night_lights_q1,night_lights_q2,night_lights_q3,"A, B, D, E. Agriculture, energy and water_q1","A, B, D, E. Agriculture, energy and water_q2","A, B, D, E. Agriculture, energy and water_q3",C. Manufacturing_q1,...,lseCWA_q3,lteOri_q1,lteOri_q2,lteOri_q3,lteWNB_q1,lteWNB_q2,lteWNB_q3,lieWCe_q1,lieWCe_q2,lieWCe_q3
id,,,,,,,,,,,,,,,,,,,,,
0,0.621362,1.003223,1.433596,0.274685,0.348078,0.693090,0.016043,0.027233,0.038466,0.014221,...,2590.133698,6.212600,6.557172,18.817714,0.000513,0.000573,0.001483,0.000688,0.001141,0.001141
4,0.689311,1.108365,1.718617,0.345601,0.941548,2.063069,0.023795,0.041716,0.080948,0.007925,...,3911.029447,14.046576,19.333049,31.125920,0.000547,0.000774,0.001502,0.000750,0.001466,0.002775
9,0.482399,0.844428,1.431697,0.175000,0.192473,0.220000,0.020990,0.036742,0.062296,0.015466,...,2416.209660,8.130102,8.130102,8.130102,0.000636,0.000636,0.000636,0.001014,0.001014,0.001014
11,0.784719,1.358327,2.271135,0.230000,0.265070,0.300054,0.029100,0.050371,0.084220,0.021442,...,586.260502,22.166346,29.197871,35.469163,0.000774,0.002636,0.003532,0.000030,0.000950,0.004667
14,0.546802,1.164578,1.987426,0.253550,0.344493,0.469610,0.027294,0.063200,0.125081,0.016771,...,4501.147469,18.572602,35.110274,38.917205,0.000058,0.001161,0.001542,0.000022,0.000614,0.001049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94815,0.892099,14.707298,50.250385,27.949167,29.960000,37.430000,0.002751,0.042119,0.169976,0.023250,...,655.029837,18.239818,25.875325,27.897271,0.008162,0.009350,0.017333,0.000024,0.000049,0.001019
94853,0.452811,2.449458,8.025306,26.741250,35.080002,42.660002,0.002557,0.020820,0.118989,0.101416,...,2463.258971,12.094334,23.690112,26.016469,0.000638,0.004672,0.015047,0.000564,0.001024,0.003049
94892,1.323585,3.714000,11.135548,16.539999,36.570000,44.580002,0.002956,0.017598,0.063195,0.060149,...,1642.327127,11.523722,17.665056,26.331681,0.002315,0.004635,0.010368,0.000207,0.000839,0.001229


## Rename columns

In [11]:
column_names = {
 'population_q1': "Population (Q1)",
 'population_q2': "Population (Q2)",
 'population_q3': "Population (Q3)",
 'night_lights_q1': "Night lights (Q1)",
 'night_lights_q2': "Night lights (Q2)",
 'night_lights_q3': "Night lights (Q3)",
 'A, B, D, E. Agriculture, energy and water_q1': "Workplace population [Agriculture, energy and water] (Q1)",
 'A, B, D, E. Agriculture, energy and water_q2': "Workplace population [Agriculture, energy and water] (Q2)",
 'A, B, D, E. Agriculture, energy and water_q3': "Workplace population [Agriculture, energy and water] (Q3)",
 'C. Manufacturing_q1': "Workplace population [Manufacturing] (Q1)",
 'C. Manufacturing_q2': "Workplace population [Manufacturing] (Q2)",
 'C. Manufacturing_q3': "Workplace population [Manufacturing] (Q3)",
 'F. Construction_q1': "Workplace population [Construction] (Q1)",
 'F. Construction_q2': "Workplace population [Construction] (Q2)",
 'F. Construction_q3': "Workplace population [Construction] (Q3)",
 'G, I. Distribution, hotels and restaurants_q1': "Workplace population [Distribution, hotels and restaurants] (Q1)",
 'G, I. Distribution, hotels and restaurants_q2': "Workplace population [Distribution, hotels and restaurants] (Q2)",
 'G, I. Distribution, hotels and restaurants_q3': "Workplace population [Distribution, hotels and restaurants] (Q3)",
 'H, J. Transport and communication_q1': "Workplace population [Transport and communication] (Q1)",
 'H, J. Transport and communication_q2': "Workplace population [Transport and communication] (Q2)",
 'H, J. Transport and communication_q3': "Workplace population [Transport and communication] (Q3)",
 'K, L, M, N. Financial, real estate, professional and administrative activities_q1': "Workplace population [Financial, real estate, professional and administrative activities] (Q1)",
 'K, L, M, N. Financial, real estate, professional and administrative activities_q2': "Workplace population [Financial, real estate, professional and administrative activities] (Q2)",
 'K, L, M, N. Financial, real estate, professional and administrative activities_q3': "Workplace population [Financial, real estate, professional and administrative activities] (Q3)",
 'O,P,Q. Public administration, education and health_q1': "Workplace population [Public administration, education and health] (Q1)",
 'O,P,Q. Public administration, education and health_q2': "Workplace population [Public administration, education and health] (Q2)",
 'O,P,Q. Public administration, education and health_q3': "Workplace population [Public administration, education and health] (Q3)",
 'R, S, T, U. Other_q1': "Workplace population [Other] (Q1)",
 'R, S, T, U. Other_q2': "Workplace population [Other] (Q2)",
 'R, S, T, U. Other_q3': "Workplace population [Other] (Q3)",
 'Code_18_124_q1': 'Land cover [124] (Q1)',
 'Code_18_124_q2': 'Land cover [124] (Q2)',
 'Code_18_124_q3': 'Land cover [124] (Q3)',
 'Code_18_211_q1': 'Land cover [211] (Q1)',
 'Code_18_211_q2': 'Land cover [211] (Q2)',
 'Code_18_211_q3': 'Land cover [211] (Q3)',
 'Code_18_121_q1': 'Land cover [121] (Q1)',
 'Code_18_121_q2': 'Land cover [121] (Q2)',
 'Code_18_121_q3': 'Land cover [121] (Q3)',
 'Code_18_421_q1': 'Land cover [421] (Q1)',
 'Code_18_421_q2': 'Land cover [421] (Q2)',
 'Code_18_421_q3': 'Land cover [421] (Q3)',
 'Code_18_522_q1': 'Land cover [522] (Q1)',
 'Code_18_522_q2': 'Land cover [522] (Q2)',
 'Code_18_522_q3': 'Land cover [522] (Q3)',
 'Code_18_142_q1': 'Land cover [142] (Q1)',
 'Code_18_142_q2': 'Land cover [142] (Q2)',
 'Code_18_142_q3': 'Land cover [142] (Q3)',
 'Code_18_141_q1': 'Land cover [141] (Q1)',
 'Code_18_141_q2': 'Land cover [141] (Q2)',
 'Code_18_141_q3': 'Land cover [141] (Q3)',
 'Code_18_112_q1': 'Land cover [112] (Q1)',
 'Code_18_112_q2': 'Land cover [112] (Q2)',
 'Code_18_112_q3': 'Land cover [112] (Q3)',
 'Code_18_231_q1': 'Land cover [231] (Q1)',
 'Code_18_231_q2': 'Land cover [231] (Q2)',
 'Code_18_231_q3': 'Land cover [231] (Q3)',
 'Code_18_311_q1': 'Land cover [311] (Q1)',
 'Code_18_311_q2': 'Land cover [311] (Q2)',
 'Code_18_311_q3': 'Land cover [311] (Q3)',
 'Code_18_131_q1': 'Land cover [131] (Q1)',
 'Code_18_131_q2': 'Land cover [131] (Q2)',
 'Code_18_131_q3': 'Land cover [131] (Q3)',
 'Code_18_123_q1': 'Land cover [123] (Q1)',
 'Code_18_123_q2': 'Land cover [123] (Q2)',
 'Code_18_123_q3': 'Land cover [123] (Q3)',
 'Code_18_122_q1': 'Land cover [122] (Q1)',
 'Code_18_122_q2': 'Land cover [122] (Q2)',
 'Code_18_122_q3': 'Land cover [122] (Q3)',
 'Code_18_512_q1': 'Land cover [512] (Q1)',
 'Code_18_512_q2': 'Land cover [512] (Q2)',
 'Code_18_512_q3': 'Land cover [512] (Q3)',
 'Code_18_243_q1': 'Land cover [243] (Q1)',
 'Code_18_243_q2': 'Land cover [243] (Q2)',
 'Code_18_243_q3': 'Land cover [243] (Q3)',
 'Code_18_313_q1': 'Land cover [313] (Q1)',
 'Code_18_313_q2': 'Land cover [313] (Q2)',
 'Code_18_313_q3': 'Land cover [313] (Q3)',
 'Code_18_412_q1': 'Land cover [412] (Q1)',
 'Code_18_412_q2': 'Land cover [412] (Q2)',
 'Code_18_412_q3': 'Land cover [412] (Q3)',
 'Code_18_321_q1': 'Land cover [321] (Q1)',
 'Code_18_321_q2': 'Land cover [321] (Q2)',
 'Code_18_321_q3': 'Land cover [321] (Q3)',
 'Code_18_322_q1': 'Land cover [322] (Q1)',
 'Code_18_322_q2': 'Land cover [322] (Q2)',
 'Code_18_322_q3': 'Land cover [322] (Q3)',
 'Code_18_324_q1': 'Land cover [324] (Q1)',
 'Code_18_324_q2': 'Land cover [324] (Q2)',
 'Code_18_324_q3': 'Land cover [324] (Q3)',
 'Code_18_111_q1': 'Land cover [111] (Q1)',
 'Code_18_111_q2': 'Land cover [111] (Q2)',
 'Code_18_111_q3': 'Land cover [111] (Q3)',
 'Code_18_423_q1': 'Land cover [423] (Q1)',
 'Code_18_423_q2': 'Land cover [423] (Q2)',
 'Code_18_423_q3': 'Land cover [423] (Q3)',
 'Code_18_523_q1': 'Land cover [523] (Q1)',
 'Code_18_523_q2': 'Land cover [523] (Q2)',
 'Code_18_523_q3': 'Land cover [523] (Q3)',
 'Code_18_312_q1': 'Land cover [312] (Q1)',
 'Code_18_312_q2': 'Land cover [312] (Q2)',
 'Code_18_312_q3': 'Land cover [312] (Q3)',
 'Code_18_133_q1': 'Land cover [133] (Q1)',
 'Code_18_133_q2': 'Land cover [133] (Q2)',
 'Code_18_133_q3': 'Land cover [133] (Q3)',
 'Code_18_333_q1': 'Land cover [333] (Q1)',
 'Code_18_333_q2': 'Land cover [333] (Q2)',
 'Code_18_333_q3': 'Land cover [333] (Q3)',
 'Code_18_332_q1': 'Land cover [332] (Q1)',
 'Code_18_332_q2': 'Land cover [332] (Q2)',
 'Code_18_332_q3': 'Land cover [332] (Q3)',
 'Code_18_411_q1': 'Land cover [411] (Q1)',
 'Code_18_411_q2': 'Land cover [411] (Q2)',
 'Code_18_411_q3': 'Land cover [411] (Q3)',
 'Code_18_132_q1': 'Land cover [132] (Q1)',
 'Code_18_331_q2': 'Land cover [331] (Q2)',
 'Code_18_222_q1': 'Land cover [222] (Q1)',
 'Code_18_511_q3': 'Land cover [511] (Q3)',
 'Code_18_242_q1': 'Land cover [242] (Q1)',
 'Code_18_511_q2': 'Land cover [511] (Q2)',
 'Code_18_242_q3': 'Land cover [242] (Q3)',
 'Code_18_331_q1': 'Land cover [331] (Q1)',
 'Code_18_334_q2': 'Land cover [334] (Q2)',
 'Code_18_511_q1': 'Land cover [511] (Q1)',
 'Code_18_334_q1': 'Land cover [334] (Q1)',
 'Code_18_222_q3': 'Land cover [222] (Q3)',
 'Code_18_242_q2': 'Land cover [242] (Q2)',
 'Code_18_244_q3': 'Land cover [244] (Q3)',
 'Code_18_521_q2': 'Land cover [521] (Q2)',
 'Code_18_334_q3': 'Land cover [334] (Q3)',
 'Code_18_244_q1': 'Land cover [244] (Q1)',
 'Code_18_244_q2': 'Land cover [244] (Q2)',
 'Code_18_331_q3': 'Land cover [331] (Q3)',
 'Code_18_132_q2': 'Land cover [132] (Q2)',
 'Code_18_132_q3': 'Land cover [132] (Q3)',
 'Code_18_521_q1': 'Land cover [521] (Q1)',
 'Code_18_222_q2': 'Land cover [222] (Q2)',
 'Code_18_521_q3': 'Land cover [521] (Q3)',
 'mean_q1': 'NDVI (Q1)',
 'mean_q2': 'NDVI (Q2)',
 'mean_q3': 'NDVI (Q3)',
 'supermarkets_nearest': "Supermarkets [distance to nearest]",
 'supermarkets_counts': "Supermarkets [counts within 1200m]",
 'listed_nearest': "Listed buildings [distance to nearest]",
 'listed_counts': "Listed buildings [counts within 1200m]",
 'fhrs_nearest': "FHRS points [distance to nearest]",
 'fhrs_counts': "FHRS points [counts within 1200m]",
 'culture_nearest': "Cultural venues [distance to nearest]",
 'culture_counts': "Cultural venues [counts within 1200m]",
 'nearest_water': "Water bodies [distance to nearest]",
 'nearest_retail_centre': "Retail centres [distance to nearest]",
 'sdbAre_q1': 'area of building (Q1)',
 'sdbAre_q2': 'area of building (Q2)',
 'sdbAre_q3': 'area of building (Q3)',
 'sdbPer_q1': 'perimeter of building (Q1)',
 'sdbPer_q2': 'perimeter of building (Q2)',
 'sdbPer_q3': 'perimeter of building (Q3)',
 'sdbCoA_q1': 'courtyard area of building (Q1)',
 'sdbCoA_q2': 'courtyard area of building (Q2)',
 'sdbCoA_q3': 'courtyard area of building (Q3)',
 'ssbCCo_q1': 'circular compactness of building (Q1)',
 'ssbCCo_q2': 'circular compactness of building (Q2)',
 'ssbCCo_q3': 'circular compactness of building (Q3)',
 'ssbCor_q1': 'corners of building (Q1)',
 'ssbCor_q2': 'corners of building (Q2)',
 'ssbCor_q3': 'corners of building (Q3)',
 'ssbSqu_q1': 'squareness of building (Q1)',
 'ssbSqu_q2': 'squareness of building (Q2)',
 'ssbSqu_q3': 'squareness of building (Q3)',
 'ssbERI_q1': 'equivalent rectangular index of building (Q1)',
 'ssbERI_q2': 'equivalent rectangular index of building (Q2)',
 'ssbERI_q3': 'equivalent rectangular index of building (Q3)',
 'ssbElo_q1': 'elongation of building (Q1)',
 'ssbElo_q2': 'elongation of building (Q2)',
 'ssbElo_q3': 'elongation of building (Q3)',
 'ssbCCM_q1': 'centroid - corner mean distance of building (Q1)',
 'ssbCCM_q2': 'centroid - corner mean distance of building (Q2)',
 'ssbCCM_q3': 'centroid - corner mean distance of building (Q3)',
 'ssbCCD_q1': 'centroid - corner distance deviation of building (Q1)',
 'ssbCCD_q2': 'centroid - corner distance deviation of building (Q2)',
 'ssbCCD_q3': 'centroid - corner distance deviation of building (Q3)',
 'stbOri_q1': 'orientation of building (Q1)',
 'stbOri_q2': 'orientation of building (Q2)',
 'stbOri_q3': 'orientation of building (Q3)',
 'sdcLAL_q1': 'longest axis length of ETC (Q1)',
 'sdcLAL_q2': 'longest axis length of ETC (Q2)',
 'sdcLAL_q3': 'longest axis length of ETC (Q3)',
 'sdcAre_q1': 'area of ETC (Q1)',
 'sdcAre_q2': 'area of ETC (Q2)',
 'sdcAre_q3': 'area of ETC (Q3)',
 'sscCCo_q1': 'circular compactness of ETC (Q1)',
 'sscCCo_q2': 'circular compactness of ETC (Q2)',
 'sscCCo_q3': 'circular compactness of ETC (Q3)',
 'sscERI_q1': 'equivalent rectangular index of ETC (Q1)',
 'sscERI_q2': 'equivalent rectangular index of ETC (Q2)',
 'sscERI_q3': 'equivalent rectangular index of ETC (Q3)',
 'stcOri_q1': 'orientation of ETC (Q1)',
 'stcOri_q2': 'orientation of ETC (Q2)',
 'stcOri_q3': 'orientation of ETC (Q3)',
 'sicCAR_q1': 'covered area ratio of ETC (Q1)',
 'sicCAR_q2': 'covered area ratio of ETC (Q2)',
 'sicCAR_q3': 'covered area ratio of ETC (Q3)',
 'stbCeA_q1': 'cell alignment of building (Q1)',
 'stbCeA_q2': 'cell alignment of building (Q2)',
 'stbCeA_q3': 'cell alignment of building (Q3)',
 'mtbAli_q1': 'alignment of neighbouring buildings (Q1)',
 'mtbAli_q2': 'alignment of neighbouring buildings (Q2)',
 'mtbAli_q3': 'alignment of neighbouring buildings (Q3)',
 'mtbNDi_q1': 'mean distance between neighbouring buildings (Q1)',
 'mtbNDi_q2': 'mean distance between neighbouring buildings (Q2)',
 'mtbNDi_q3': 'mean distance between neighbouring buildings (Q3)',
 'mtcWNe_q1': 'perimeter-weighted neighbours of ETC (Q1)',
 'mtcWNe_q2': 'perimeter-weighted neighbours of ETC (Q2)',
 'mtcWNe_q3': 'perimeter-weighted neighbours of ETC (Q3)',
 'mdcAre_q1': 'area covered by neighbouring cells (Q1)',
 'mdcAre_q2': 'area covered by neighbouring cells (Q2)',
 'mdcAre_q3': 'area covered by neighbouring cells (Q3)',
 'ltcWRE_q1': 'weighted reached enclosures of ETC (Q1)',
 'ltcWRE_q2': 'weighted reached enclosures of ETC (Q2)',
 'ltcWRE_q3': 'weighted reached enclosures of ETC (Q3)',
 'ltbIBD_q1': 'mean inter-building distance (Q1)',
 'ltbIBD_q2': 'mean inter-building distance (Q2)',
 'ltbIBD_q3': 'mean inter-building distance (Q3)',
 'sdsSPW_q1': 'width of street profile (Q1)',
 'sdsSPW_q2': 'width of street profile (Q2)',
 'sdsSPW_q3': 'width of street profile (Q3)',
 'sdsSWD_q1': 'width deviation of street profile (Q1)',
 'sdsSWD_q2': 'width deviation of street profile (Q2)',
 'sdsSWD_q3': 'width deviation of street profile (Q3)',
 'sdsSPO_q1': 'openness of street profile (Q1)',
 'sdsSPO_q2': 'openness of street profile (Q2)',
 'sdsSPO_q3': 'openness of street profile (Q3)',
 'sdsLen_q1': 'length of street segment (Q1)',
 'sdsLen_q2': 'length of street segment (Q2)',
 'sdsLen_q3': 'length of street segment (Q3)',
 'sssLin_q1': 'linearity of street segment (Q1)',
 'sssLin_q2': 'linearity of street segment (Q2)',
 'sssLin_q3': 'linearity of street segment (Q3)',
 'ldsMSL_q1': 'mean segment length within 3 steps (Q1)',
 'ldsMSL_q2': 'mean segment length within 3 steps (Q2)',
 'ldsMSL_q3': 'mean segment length within 3 steps (Q3)',
 'mtdDeg_q1': 'node degree of junction (Q1)',
 'mtdDeg_q2': 'node degree of junction (Q2)',
 'mtdDeg_q3': 'node degree of junction (Q3)',
 'lcdMes_q1': 'local meshedness of street network (Q1)',
 'lcdMes_q2': 'local meshedness of street network (Q2)',
 'lcdMes_q3': 'local meshedness of street network (Q3)',
 'linP3W_q1': 'local proportion of 3-way intersections of street network (Q1)',
 'linP3W_q2': 'local proportion of 3-way intersections of street network (Q2)',
 'linP3W_q3': 'local proportion of 3-way intersections of street network (Q3)',
 'linP4W_q1': 'local proportion of 4-way intersections of street network (Q1)',
 'linP4W_q2': 'local proportion of 4-way intersections of street network (Q2)',
 'linP4W_q3': 'local proportion of 4-way intersections of street network (Q3)',
 'linPDE_q1': 'local proportion of cul-de-sacs of street network (Q1)',
 'linPDE_q2': 'local proportion of cul-de-sacs of street network (Q2)',
 'linPDE_q3': 'local proportion of cul-de-sacs of street network (Q3)',
 'lcnClo_q1': 'local closeness of street network (Q1)',
 'lcnClo_q2': 'local closeness of street network (Q2)',
 'lcnClo_q3': 'local closeness of street network (Q3)',
 'ldsCDL_q1': 'local cul-de-sac length of street network (Q1)',
 'ldsCDL_q2': 'local cul-de-sac length of street network (Q2)',
 'ldsCDL_q3': 'local cul-de-sac length of street network (Q3)',
 'xcnSCl_q1': 'square clustering of street network (Q1)',
 'xcnSCl_q2': 'square clustering of street network (Q2)',
 'xcnSCl_q3': 'square clustering of street network (Q3)',
 'mtdMDi_q1': 'mean distance to neighbouring nodes of street network (Q1)',
 'mtdMDi_q2': 'mean distance to neighbouring nodes of street network (Q2)',
 'mtdMDi_q3': 'mean distance to neighbouring nodes of street network (Q3)',
 'lddNDe_q1': 'local node density of street network (Q1)',
 'lddNDe_q2': 'local node density of street network (Q2)',
 'lddNDe_q3': 'local node density of street network (Q3)',
 'linWID_q1': 'local degree weighted node density of street network (Q1)',
 'linWID_q2': 'local degree weighted node density of street network (Q2)',
 'linWID_q3': 'local degree weighted node density of street network (Q3)',
 'stbSAl_q1': 'street alignment of building (Q1)',
 'stbSAl_q2': 'street alignment of building (Q2)',
 'stbSAl_q3': 'street alignment of building (Q3)',
 'sddAre_q1': 'area covered by node-attached ETCs (Q1)',
 'sddAre_q2': 'area covered by node-attached ETCs (Q2)',
 'sddAre_q3': 'area covered by node-attached ETCs (Q3)',
 'sdsAre_q1': 'area covered by edge-attached ETCs (Q1)',
 'sdsAre_q2': 'area covered by edge-attached ETCs (Q2)',
 'sdsAre_q3': 'area covered by edge-attached ETCs (Q3)',
 'sisBpM_q1': 'buildings per meter of street segment (Q1)',
 'sisBpM_q2': 'buildings per meter of street segment (Q2)',
 'sisBpM_q3': 'buildings per meter of street segment (Q3)',
 'misCel_q1': 'reached ETCs by neighbouring segments (Q1)',
 'misCel_q2': 'reached ETCs by neighbouring segments (Q2)',
 'misCel_q3': 'reached ETCs by neighbouring segments (Q3)',
 'mdsAre_q1': 'reached area by neighbouring segments (Q1)',
 'mdsAre_q2': 'reached area by neighbouring segments (Q2)',
 'mdsAre_q3': 'reached area by neighbouring segments (Q3)',
 'lisCel_q1': 'reached ETCs by local street network (Q1)',
 'lisCel_q2': 'reached ETCs by local street network (Q2)',
 'lisCel_q3': 'reached ETCs by local street network (Q3)',
 'ldsAre_q1': 'reached area by local street network (Q1)',
 'ldsAre_q2': 'reached area by local street network (Q2)',
 'ldsAre_q3': 'reached area by local street network (Q3)',
 'ltcRea_q1': 'reached ETCs by tessellation contiguity (Q1)',
 'ltcRea_q2': 'reached ETCs by tessellation contiguity (Q2)',
 'ltcRea_q3': 'reached ETCs by tessellation contiguity (Q3)',
 'ltcAre_q1': 'reached area by tessellation contiguity (Q1)',
 'ltcAre_q2': 'reached area by tessellation contiguity (Q2)',
 'ltcAre_q3': 'reached area by tessellation contiguity (Q3)',
 'ldeAre_q1': 'area of enclosure (Q1)',
 'ldeAre_q2': 'area of enclosure (Q2)',
 'ldeAre_q3': 'area of enclosure (Q3)',
 'ldePer_q1': 'perimeter of enclosure (Q1)',
 'ldePer_q2': 'perimeter of enclosure (Q2)',
 'ldePer_q3': 'perimeter of enclosure (Q3)',
 'lseCCo_q1': 'circular compactness of enclosure (Q1)',
 'lseCCo_q2': 'circular compactness of enclosure (Q2)',
 'lseCCo_q3': 'circular compactness of enclosure (Q3)',
 'lseERI_q1': 'equivalent rectangular index of enclosure (Q1)',
 'lseERI_q2': 'equivalent rectangular index of enclosure (Q2)',
 'lseERI_q3': 'equivalent rectangular index of enclosure (Q3)',
 'lseCWA_q1': 'compactness-weighted axis of enclosure (Q1)',
 'lseCWA_q2': 'compactness-weighted axis of enclosure (Q2)',
 'lseCWA_q3': 'compactness-weighted axis of enclosure (Q3)',
 'lteOri_q1': 'orientation of enclosure (Q1)',
 'lteOri_q2': 'orientation of enclosure (Q2)',
 'lteOri_q3': 'orientation of enclosure (Q3)',
 'lteWNB_q1': 'perimeter-weighted neighbours of enclosure (Q1)',
 'lteWNB_q2': 'perimeter-weighted neighbours of enclosure (Q2)',
 'lteWNB_q3': 'perimeter-weighted neighbours of enclosure (Q3)',
 'lieWCe_q1': 'area-weighted ETCs of enclosure (Q1)',
 'lieWCe_q2': 'area-weighted ETCs of enclosure (Q2)',
 'lieWCe_q3': 'area-weighted ETCs of enclosure (Q3)',
}

Get land cover classes.

In [7]:
clc_legend = pandas.read_csv("../../urbangrammar_samba/functional_data/land_use/corine/u2018_clc2018_v2020_20u1_geoPackage/Legend/CLC_legend.csv", delimiter=";")

In [10]:
clc_key = dict(zip(clc_legend.CLC_CODE, clc_legend.LABEL3))

In [26]:
for i, key in column_names.items():
    if "Land cover" in key:
        column_names[i] = key[:12] + clc_key[int(key[12:15])] + key[15:]

In [31]:
column_names

{'population_q1': 'Population (Q1)',
 'population_q2': 'Population (Q2)',
 'population_q3': 'Population (Q3)',
 'night_lights_q1': 'Night lights (Q1)',
 'night_lights_q2': 'Night lights (Q2)',
 'night_lights_q3': 'Night lights (Q3)',
 'A, B, D, E. Agriculture, energy and water_q1': 'Workplace population [Agriculture, energy and water] (Q1)',
 'A, B, D, E. Agriculture, energy and water_q2': 'Workplace population [Agriculture, energy and water] (Q2)',
 'A, B, D, E. Agriculture, energy and water_q3': 'Workplace population [Agriculture, energy and water] (Q3)',
 'C. Manufacturing_q1': 'Workplace population [Manufacturing] (Q1)',
 'C. Manufacturing_q2': 'Workplace population [Manufacturing] (Q2)',
 'C. Manufacturing_q3': 'Workplace population [Manufacturing] (Q3)',
 'F. Construction_q1': 'Workplace population [Construction] (Q1)',
 'F. Construction_q2': 'Workplace population [Construction] (Q2)',
 'F. Construction_q3': 'Workplace population [Construction] (Q3)',
 'G, I. Distribution, hotels

Assing new column names

In [30]:
per_type = per_type.rename(columns=column_names)

In [46]:
per_type.to_csv("per_type.csv")

In [ ]:
per_geometry = per_geometry.rename(columns=column_names)

In [47]:
per_geometry.to_csv("per_geometry.csv")